### HydroGen

Generalization of hydraulic properties from BioSoil data (MaaTi WP4.2)

A.-J. Kieloaho, J. Heiskanen, and S. Launiainen

0) Get access and familiarize to Biosoil and ICP L2 soil data + get all data!

1) Re-fit WRC models to data using larger parameter ranges (Bittelli, Soil Physics with Python):
    * vanGenuchten
    * brooks-corey
    * campbell

2) Generic WRC curves / profiles:
    * classical way: group per site type etc., form average curves 
    * use clustering algorithm --> N generic curves, assign to

3) Upscaling to space (regression models, ML-models) by open gis-data
    * WRC curves
    * organic layer depth
    * other?

4) Write a short paper describing data, clustering and upscaling. Data product supplement

5) Include simple demonstration on effect of hydraulic properties: soil moisture dynamics & drougth risk levels

***

Data:
- Biosoil_Maafysik_Koealoittain_2006_07.xls
- Biosoil_VedenpidätysTulokset2006_07.xls



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from MaaTi_io import read_data, preprocessing_data

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Reading data
# xlsx.table.sheet -> pandas.DataFrame
# header is row 0
# in xlsx.table.sheet there is orphan data cells -> remove

# columns are read in and labels are renamed 

import pandas as pd
import numpy as np

file_path = 'Data/Biosoil/'
file_name = 'Biosoil_Maafysik_Koealoittain_2006_07_test.xlsx'
sheet_name = 'DataByPlots'

data = read_data(file_path=file_path, file_name=file_name, sheet_name=sheet_name)
data = preprocessing_data(data=data)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
print(data.columns)

data.to_csv('preprocessed_BioSoil_MaaTi.csv')

Index(['depth', 'site_id', 'year', 'month', 'day', 'lat_N', 'lon_E',
       'altitude', 'soil_type', 'grain_size', 'orglayer_thickness',
       'topography', 'slope', 'hydraulic_regime', 'peat_forming_mosses',
       'sitetype', 'vmi_horizon', 'clay', 'silt', 'sand', 'raw_bulk_density',
       'gravel', 'dry_density', 'organic_content', 'raw_vwc_03kPa',
       'raw_vwc_1kPa', 'raw_vwc_5kPa', 'raw_vwc_10kPa', 'raw_vwc_100kPa',
       'disturbance_flag', 'mass_stones', 'vol_stones', 'vwc_001kPa',
       'vwc_03kPa', 'vwc_1kPa', 'vwc_5kPa', 'vwc_10kPa', 'vwc_100kPa',
       'vwc_1500kPa', 'bulk_density', 'bhor', 'afp_10kPa', 'raw_paw', 'paw',
       'org+clay_ratio'],
      dtype='object')


In [3]:
def wrc_functions(model):
    
    if model == 'Campbell':
        #return lambda h, *p: p[0] * (h / p[1]) **(-1. / p[2])
    
        def cb(h, *p):
            eff_sat = (h * p[1]) **(-1. / p[2])
            
            theta = eff_sat * p[0]
            theta[h <= p[1]] = p[0]
        
            return theta
        
        return cb

    elif model == 'vanGenuchten_2par':
        #return lambda h, alpha, n: 1 / (1.0 + (alpha * abs(h))**n)**(1.0 - 1.0 / n)
        
        def vg_2par(h, alpha, n):
            eff_sat = 1.0 / (1.0 + (alpha * abs(h))**n)**(1.0 - 1.0 / n)
            
            theta = eff_sat * (theta_s - theta_r) + theta_r
            
            return theta
            
            #return eff_sat
        
        return vg_2par
    
    elif model == 'vanGenuchten_3par':
        #return lambda h, alpha, n, m: 1 / (1.0 + (alpha * abs(h)**n)**m)
        
        def vg_3par(h, theta_s, theta_r, alpha, n,):
            
            eff_sat = 1.0 / (1.0 + (alpha * abs(h))**n)**(1.0 - 1.0 / n)
            theta = eff_sat * (theta_s - theta_r) + theta_r
            
            
            return theta
        
        return vg_3par

    elif model == 'vanGenuchten_4par':
        #return lambda h, *p:   p[1] + (p[0] - p[1]) / (1. + (p[2] * h) **p[3]) **(1. - 1. / p[3])
        
        def vg_4par(h, theta_s, theta_r, alpha, n):
            m = 1. - (1. / n)
            eff_sat = 1. / (1. + (alpha * abs(h))**n)**m
            
            #eff_sat[eff_sat > 1.0] = 1.0
            #eff_sat[eff_sat < 0.0] = 0.0
            
            theta = eff_sat * (theta_s - theta_r) + theta_r
            
            return theta
        
        return vg_4par

    
print('Done!')

Done!


In [4]:
# Calculation of water retention curves
from scipy.optimize import curve_fit
from lmfit import Model, Parameters
from PSP_waterRetentionFitting.PSP_Marquardt import *
from PSP_waterRetentionFitting.PSP_waterRetention import Campbell, VanGenuchten, VanGenuchtenRestricted
from PSP_waterRetentionFitting.PSP_waterRetention import IppischVanGenuchten, CampbellIppischVanGenuchten
eps = np.finfo(float).eps  # machine epsilon


regex = ('^(?!raw_)vwc_\d{1,4}kPa$')
cols_wrc = data.filter(regex=regex).columns.tolist()

hydraulic_potentials = np.array([0.01, 0.3, 1.0, 5.0, 10.0, 100.0, 1500.0]) #* 10.197  # kPa -> cm H2O
cols_to_kPa = {
    'vwc_001kPa': 0.01,
    'vwc_03kPa': 0.3,
    'vwc_1kPa': 1.0,
    'vwc_5kPa': 5.0,
    'vwc_10kPa': 10.0,
    'vwc_100kPa': 100.0,
    'vwc_1500kPa': 1500.0
}

# 1. depth-wise
# groupby depth and calculate means
# (1=0-6cm, 2=15-21cm, 3=30-36cm, 4=0-6cm (B-hor)
#display(data)

flags_to_depth = {
    1: '0-6 cm (A hor)',
    2: '15-21 cm',
    3: '30-36 cm',
    4: '0-6 cm (B hor)'
}

wrc_data_depthwise = data.groupby(['depth'])[cols_wrc].mean()
#display(wrc_data_depthwise)

#for i in range(len(wrc_data_depthwise)):
#    plt.semilogy(wrc_data_depthwise.iloc[i,:], hydraulic_potentials, '.', label=flags_to_depth[i+1])
#    plt.legend(frameon=False)

#for key, value in wrc_depth_means.items():
#    plt.semilogy(value.values, hydraulic_potentials)

# cambell: 0. theta_s, 1. he, 2. b
# vg_5par: 0. theta_s, 1. theta_r, 2. alpha, 3. n, 4. m
# vg_4par: 0. theta_s, 1. theta_r, 2. alpha, 3. n, 4. m
# ippi_vg: 0. theta_s, 1. theta_r, 2. he, 3. alpha, 4. n
# caippvg: 0. theta_s, 1. theta_r, 2. he, 3. alpha, 4. n

wrc_data = {
    'vanGenuchten_2par': pd.DataFrame(np.NaN, index=data.index, columns=['theta_s', 'theta_r', 'alpha', 'n', 'theta_ratio', 'chi_sqr', 'reduced_chi_sqr', 'aic', 'bic']),  # van Genuchten with the restriction m = 1 - 1/n
    'vanGenuchten_3par': pd.DataFrame(np.NaN, index=data.index, columns=['theta_s', 'theta_r', 'alpha', 'n', 'theta_ratio', 'chi_sqr', 'reduced_chi_sqr', 'aic', 'bic']),  # van Genuchten
    'vanGenuchten_4par': pd.DataFrame(np.NaN, index=data.index, columns=['theta_s', 'theta_r', 'alpha', 'n', 'theta_ratio', 'chi_sqr', 'reduced_chi_sqr', 'aic', 'bic']),  # van Genuchten with the restriction m = 1 - 1/n
}

wrc_results = {i: {} for i in data.index}



# initial_values
air_entry = 1.0
campbell_b = 4.0
alpha_vg = 1.0 / air_entry
n_vg = 1.5
m_vg = 1.5 - 1.5 / n_vg

for i in data.index:
    
    # select volumetric water contents and convert from percentages to m3 m-3 
    
    for model_name in wrc_data:
        
        vwc = np.array(data.loc[i, list(cols_to_kPa.keys())] / 100.0)

        theta_s = max(vwc)
        theta_r = min(vwc)
            
        if model_name == 'vanGenuchten_2par':
        # Note: vanGenuchten_2par does return EFFECTIVE SATURATION not THETA
            #vwc = (vwc - theta_r) / (theta_s - theta_r) + eps
            
            # parameters are listed as tuples containing following information
            # (name, value, vary, min, max, expr, brute_step)
            param_list = [
                ('theta_s', theta_s, False),
                ('theta_r', theta_r, False),
                ('alpha', alpha_vg, True, 0.001, 200., None, None),
                ('n', n_vg, True, 0.01, 200.0, None, None)
            ]
            
        elif model_name == 'vanGenuchten_3par':
        
            # parameters are listed as tuples containing following information
            # (name, value, vary, min, max, expr, brute_step)
            param_list = [
                ('theta_s', theta_s, False),
                ('theta_r', theta_r, True, 0.0, theta_r, None, None),
                ('alpha', alpha_vg, True, 0.001, 200., None, None),
                ('n', n_vg, True, 0.01, 200.0, None, None),
            ]
        
        elif model_name == 'vanGenuchten_4par':
            
            # parameters are listed as tuples containing following information
            # (name, value, vary, min, max, expr, brute_step)
            param_list = [
                ('theta_s', theta_s, True, theta_r, min(1.0, theta_s*1.05), None, None),
                ('theta_r', theta_r, True, 0.0, theta_r, None, None),
                ('alpha', alpha_vg, True, 0.001, 200., None, None),
                ('n', n_vg, True, 0.01, 200.0, None, None),
            ]

        vwc = vwc.astype('float64')
        ix = np.where(vwc > 0)[0]
        
        wrc_model = Model(wrc_functions(model_name))
        wrc_params = Parameters()
        
        wrc_params.add_many(*param_list)
        
        results = wrc_model.fit(
            vwc[ix],
            wrc_params,
            h=hydraulic_potentials[ix],
            nan_policy='omit'
        )
        
        wrc_results[i][model_name] = results
        
        params = list(results.best_values.values())
        
        residuals = results.residual
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum(vwc - np.mean(vwc))**2
        mse = np.mean(residuals)
        rmse = (ss_res / (residuals.size - 2))**0.5
        r2 = 1.0 - ss_res / ss_tot
        
        if model_name == 'vanGenuchten_2par':
            params = [theta_s, theta_r] + params
            theta_ratio = theta_r / theta_s
            params.append(theta_ratio)
        
        elif model_name == 'vanGenuchten_3par':
            theta_ratio = params[1] / params[0]
            params.append(theta_ratio)
        
        elif model_name == 'vanGenuchten_4par':
            theta_ratio = params[1] / params[0]
            params.append(theta_ratio)
        
        
        params = params + [results.chisqr, results.redchi, results.aic, results.bic]
        
        #print(params)
        #params.append(rmse)
        
        wrc_data[model_name].loc[i, :] = params  # water retention function parameters
        


data_for_figs = {}

for i in data.index:
    gof = [(key, value.bic, value.aic, value.redchi) for key, value in wrc_results[i].items()]
    
    #print(wrc_results[i]['vanGenuchten_2par'].params)
    
    params = [(key, value.params['alpha'].value, value.params['n'].value, value.params['theta_r'].value, value.params['theta_s'].value) for key, value in wrc_results[i].items()]
    data_for_figs[i] = {'goodness_of_fit': gof, 'wrc': params}

#print(wrc_results['vanGenuchten_2par'])
# [62, ('vanGenuchten_2par', 8.450474262120188, 1.2301937390187345),
#('vanGenuchten_3par', 1.0000000000000016, 1.4999999999999944),
#('vanGenuchten_4par', 10.619080855558405, 1.1291768513398484)]
print('Done!')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:139: RuntimeWarning: divide by zero encountered in double_scalars


Done!


In [ ]:
# we concentrate on restricted vanGenuchten model parameters
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('20210628_MaaTi_BioSoil_wrc-fits.pdf')

psi_range = np.logspace(-2, 4., 100)
nvwc_function = lambda h, alpha, n: 1 / (1.0 + (alpha * abs(h))**n)**(1.0 - 1.0 / n)

def draw_figure_all(psi, model):

    for i in wrc[model].index:
        alpha = wrc[model].loc[i, 'alpha']
        n = wrc[model].loc[i, 'n']
    
        nvwc = nvwc_function(psi, alpha, n)
    
        plt.semilogx(psi, nvwc)
    
        
#fig = plt.figure()
#draw_figure_all(psi_range, 'vanGenuchten_2par')

#fig = plt.figure()
#draw_figure_all(psi_range, 'vanGenuchten_3par')

#fig = plt.figure()
#draw_figure_all(psi_range, 'vanGenuchten_4par')


fig_texts = {
    'vanGenuchten_2par': '$\\alpha$, n',
    'vanGenuchten_3par': '$\\theta_r$, $\\alpha$, n',
    'vanGenuchten_4par': '$\\theta_s$, $\\theta_r$, $\\alpha$, n'
}

def draw_figure_single(psi, index):
    
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    
    wrc = data_for_figs[index]['wrc']
    gof = data_for_figs[index]['goodness_of_fit']
    
    vwc = np.array(data.loc[index, list(cols_to_kPa.keys())] / 100.0)
    
    colors = ['blue', 'orange', 'green']
    
    for i, item in enumerate(wrc):
    
        model_name = item[0]
        alpha = item[1]
        n = item[2]
        theta_r = item[3]
        theta_s = item[4]
        
        redchi = gof[i][-1]
        bic = gof[i][1]
        
        #label = fig_texts[model_name]+' $\\chi^2_{reduced,naive}$ %0.4f' %redchi
        label = fig_texts[model_name]+' -- BIC %0.4f' %bic
        
        nvwc = nvwc_function(psi, alpha, n)
        
        nvwc_points = (vwc - theta_r) / (theta_s - theta_r + eps) 
        
        ax.semilogx(psi, nvwc, label=label, c=colors[i])
        ax.semilogx(hydraulic_potentials, nvwc_points, 'o', c=colors[i])
        
    ax.legend(title='Sample index in df {}'.format(index), frameon=False)
    
    
    
    ax.set_xlabel('-log($\\psi$), [m]')
    ax.set_ylabel('Effective saturation [-]')
    
for i in data.index:
    draw_figure_single(psi_range, i)
    pp.savefig()

    
pp.close()
print('Done!')

In [6]:
# save data with alpha_vg2, alpha_vg3, n_vg2, n_vg3, theta_r_vg3, theta_s, theta_ratio
# 'theta_s', 'theta_r', 'alpha', 'n', 'theta_ratio', 'chi_sqr', 'reduced_chi_sqr', 'aic', 'bic'

#print(wrc_data['vanGenuchten_2par'])

df_vg2par = wrc_data['vanGenuchten_2par']

cols_renamed = {
    'alpha': 'alpha_vg2',
    'n': 'n_vg2',
    'theta_s': 'theta-s',
    'theta_ratio': 'theta-ratio',
    'reduced_chi_sqr': 'redchi2_vg2', 
    'bic': 'bic_vg2'
}

df_vg2par = df_vg2par.rename(columns=cols_renamed)
df_vg2par = df_vg2par[cols_renamed.values()]

df_vg3par = wrc_data['vanGenuchten_3par']

cols_renamed = {
    'alpha': 'alpha_vg3',
    'n': 'n_vg3',
    'theta_r': 'theta-r_vg3',
    'reduced_chi_sqr': 'red-chi2_vg3', 
    'bic': 'bic_vg3'
}

df_vg3par = df_vg3par.rename(columns=cols_renamed)
df_vg3par = df_vg3par[cols_renamed.values()]

data = pd.concat([data, df_vg2par, df_vg3par], axis=1)

data.head(5)

,depth,site_id,year,month,day,lat_N,lon_E,altitude,soil_type,grain_size,orglayer_thickness,topography,slope,hydraulic_regime,peat_forming_mosses,sitetype,vmi_horizon,clay,silt,sand,raw_bulk_density,gravel,dry_density,organic_content,raw_vwc_03kPa,raw_vwc_1kPa,raw_vwc_5kPa,raw_vwc_10kPa,raw_vwc_100kPa,disturbance_flag,mass_stones,vol_stones,vwc_001kPa,vwc_03kPa,vwc_1kPa,vwc_5kPa,vwc_10kPa,vwc_100kPa,vwc_1500kPa,bulk_density,bhor,afp_10kPa,raw_paw,paw,org+clay_ratio,alpha_vg2,n_vg2,theta-s,theta-ratio,redchi2_vg2,bic_vg2,alpha_vg3,n_vg3,theta-r_vg3,red-chi2_vg3,bic_vg3
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
62,3,23531,2006,7,25,6809705,3485001,90,3,1,0.6,3,2,3,0,2,203,1.5,34.1,64.4,1.098,6.9717,2.62509,2.16571,100,100,100,100,100,0,NaN,NaN,54.566429,42.260327,40.673223,34.305682,26.395654,24.904158,14.106777,1.19267,0,28.1708,12.2889,27.2832,3.66571,8.450474,1.230194,0.545664,0.258525,0.001079,-46.286159,1.000000,1.500000,1.410678e-01,0.004766,-35.503668
66,3,23572,2006,7,26,6810115,3517000,120,3,3,3.25,3,5,3,0,3,203,2.8,33.4,63.7,0.996,16.9775,2.57288,6.70632,100,100,100,99.1653,100,0,NaN,NaN,58.711281,52.092829,48.285785,40.990035,30.426463,15.464652,1.784596,1.06231,0,28.2848,28.6419,29.3556,9.50632,0.683450,1.350556,0.587113,0.030396,0.001613,-43.472620,1.018337,1.265652,1.747916e-03,0.002182,-40.970660
75,4,25511,2006,7,25,6825701,3469014,120,4,3,3.3,1,5,3,0,4,None,None,None,None,None,None,2.6178,2.79994,100,97.4227,97.4227,97.7663,96.9072,0,NaN,NaN,54.562786,38.630451,32.853114,19.744303,13.825066,9.385639,0.548901,1.18946,0,40.7377,13.2762,27.2814,4.39994,6.670341,1.306416,0.545628,0.010060,0.000561,-50.858029,6.864990,1.297330,8.341784e-14,0.000657,-49.376010
78,3,25571,2006,8,5,6825693,3517009,90,3,1,0.35,3,9,3,0,2,203,2.4,56.6,41,1.138,7.03749,2.61689,2.87952,100,100,100,100,100,0,NaN,NaN,53.499098,49.178751,41.755469,38.550704,34.221002,4.554036,7.603836,1.21688,0,19.2781,26.6172,26.7495,5.27952,0.240353,1.662733,0.534991,0.085124,0.003550,-37.949763,1.154542,1.239415,2.280656e-03,0.005606,-34.367096
82,3,25591,2006,8,4,6825701,3532994,90,3,2,9.7,2,8,4,3,3,203,1.3,13.7,84.9,1.013,15.4902,2.63224,1.54474,100,97.9203,98.7868,99.1334,100,0,NaN,NaN,46.772246,42.350831,27.819910,23.244735,15.752546,3.448302,5.749630,1.40108,0,31.0197,10.0029,23.3861,2.84474,2.528521,1.403332,0.467722,0.073725,0.001114,-46.059599,2.792309,1.346617,1.105256e-02,0.001330,-44.438910


In [8]:
data.to_csv('wrc-calculated_BioSoil_Maati.csv')